In [2]:
from urllib import request 
import logging
from pathlib import Path
import numpy as np
try:
    if get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
        print("notebook")
        from tqdm import tqdm_notebook as tqdm
    else:
        raise RuntimeError
except (NameError, RuntimeError):
    from tqdm import tqdm
import re
import MeCab
from gensim import corpora, models

notebook


In [3]:
mecab = MeCab.Tagger("-Ochasen -d /usr/local/lib/mecab/dic/mecab-ipadic-neologd/")

In [4]:
res = request.urlopen("http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt")
stopwords = [line.decode("utf-8").strip() for line in res]

In [5]:
def tokenizer(text):
    l = [line.split("\t") for line in mecab.parse(text).split("\n")]
    res = [i[0] for i in l 
                   if len(i) >=4 
                       and ("名詞" in i[3] or "動詞" in i[3] or "形容詞" in i[3] )
                       and "数" not in i[3] and "助動詞" not in i[3] 
                       #and not re.search("[0-9]", i[0])
                       and i[0] not in stopwords
            ]
    return res

In [6]:
tokenizer("認めたくないものだな。自分自身の若さ故の過ちというものを。")

['認め', '自分自身', '若さ故の過ち']

In [7]:
doc_path = "./text/"
doc_dir = Path(doc_path)
dirs = [i for i in doc_dir.iterdir() if i.is_dir()]
dirs

[PosixPath('text/movie-enter'),
 PosixPath('text/it-life-hack'),
 PosixPath('text/kaden-channel'),
 PosixPath('text/topic-news'),
 PosixPath('text/livedoor-homme'),
 PosixPath('text/peachy'),
 PosixPath('text/sports-watch'),
 PosixPath('text/dokujo-tsushin'),
 PosixPath('text/smax')]

In [8]:
articles = [a for categ in dirs for a in categ.iterdir()]

In [9]:
len(articles)

7376

In [10]:
articles = articles[:30]

In [11]:
def read_doc(doc_id):
    with articles[doc_id].open() as f:
        print(f.read())

In [12]:
class Doc_manager():
    def __init__(self, docs):
        self.docs = docs
        
    def read_doc(self, doc_id):
        with self.docs[doc_id].open() as f:
            print(f.read())

In [13]:
dm = Doc_manager(articles)

In [14]:
dm.read_doc(0)

http://news.livedoor.com/article/detail/5978741/
2011-10-30T10:15:00+0900
【DVDエンター！】誘拐犯に育てられた女が目にした真実は、孤独か幸福か
　2005年11月から翌2006年7月まで読売新聞にて連載された、直木賞作家・角田光代による初の長編サスペンス『八日目の蝉』。2010年に檀れいと北乃きいの出演によりテレビドラマ化された同作が、2011年4月に永作博美と井上真央の出演によって映画化。そして、劇場公開から半年が過ぎた10月28日、DVD＆ブルーレイとなって発売されました。

八日目の蝉
　妻子ある男と愛し合い、その子を身ごもりながら、あきらめざるをえなかった女。彼女は同時に、男の妻が子供を産んだことを知る。その赤ん坊を見に行った女は、突発的にその子を連れ去り、逃避行を続けた挙句、小豆島に落ち着き、母と娘として暮らしはじめる。


不倫相手の子供を誘拐し、4年間育てた女
　永作博美が演じる野々宮希和子は、不倫相手の子を宿しながらも、彼の「いずれ妻と別れるから、それまで待ってくれ」という常套句を信じて、中絶。後遺症により、二度と子供を産めない身体となってしまいます。その後、不倫相手から彼の妻が出産したことを知らされ、別れを決意。最後に諦めをつけるため、彼らの生後6ヶ月の赤ん坊・恵理菜の顔を見た希和子でしたが、自分に笑顔で向けた恵理菜を見て、思わず誘拐。名前を変えて恵理菜を薫と名付けると、人目を避けて各地を転々とし、二人で幸せな時間を過ごしますが、辿り着いた最後の場所・小豆島で4年の逃避行に終止符を打ちます。


誘拐犯に育てられた女
　4歳になって実の両親と再会を果たした後も、世間から言われの無い中傷を受け、本当の両親との関係を築けないまま、21歳の大学生へと成長した恵理菜。過去と向き合うことを避けてきた恵理菜でしたが、劇団ひとりが演じる不倫相手・岸田孝史の子を宿し、ずっと憎み続けてきた希和子と同じ道を歩んでいることに気付いた彼女は、小池栄子が演じるルポライター・安藤千草と共に、4年間の逃亡生活を追憶する旅に出ます。希和子との幸せだった時間に触れながら、最終地・小豆島に辿り着いた恵理菜が見た真実とは？


八日目の蝉は幸せなのだろうか？
　蝉は俗説として、一生の大半を幼虫として地下で費やし

In [15]:
docs = []
for a in tqdm(articles):
    with a.open() as f:
        f.readline()
        f.readline()
        docs.append(tokenizer(f.read()))

HBox(children=(IntProgress(value=0, max=30), HTML(value='')))

In [16]:
dictionary = corpora.Dictionary(docs)

In [17]:
dictionary

In [18]:
#d.filter_extremes(no_below=5, no_above=0.2)
# nob_below以下の個数の単語を無視
# no_aboveの割合以上に出てくる単語を無視

#d.compactify()
# idを振り直してコンパクトにする。

In [19]:
dic_num = len(dictionary)
dic_num

3952

In [20]:
dictionary.doc2bow(["パソコン","テレビ","パソコン"])

[(2196, 1)]

In [21]:
corpus = [dictionary.doc2bow(w) for w in docs]

In [22]:
len(corpus)

30

In [23]:
M = len(corpus)
V =  len(dictionary)
k = 3

In [24]:
gamma_di = 1.0 
lambda_wi =  1.0

In [25]:
gamma_ = np.random.rand(M, k)
lambda_ = np.random.rand(V, k)
q_ = np.random.rand(M,V,k)

In [26]:
from scipy.special import digamma

In [ ]:
for _ in tqdm(range(1000)):
    for d in range(M):
        doc = corpus[d]
        N_d = len(doc)
        for n in range(N_d):
            gamma_sum = gamma_.sum(axis=1)
            lambda_sum = lambda_.sum(axis=0)
            w = int(doc[n][0])
            for i in range(k):
                gamma_di = gamma_[d, i]
                lambda_wi = lambda_[w, i]
                q_[d,w,i] = np.exp(digamma(gamma_di) - digamma(gamma_sum)[i] + digamma(lambda_wi) - digamma(lambda_sum)[i])
                gamma_[d,i] += q_.sum(axis=1)[d,i]
                lambda_[w,i] += q_.sum(axis=0)[w,i]
    alpha = gamma_ - np.sum(q_,axis=1)
    eta = lambda_ - np.sum(q_,axis=0)
    log_per = 0.0
    for _ in range(100):
        theta = np.array([np.random.dirichlet(a) for a in alpha])
        beta = np.array([np.random.dirichlet(e) for e in eta])
        p_w = np.inner(theta, beta)
        log_per += np.log(p_w).sum()
    log_per = log_per/100
    print(log_per)


HBox(children=(IntProgress(value=0, max=1000), HTML(value='')))

-130520.407875
-130191.758061
-130211.584266
-130222.289299
-130229.265863
-130233.243179
-130236.079223
-130238.648031
-130240.266225
-130241.495751
-130242.547619
-130243.355586
-130243.961771
-130244.592355
-130245.078356
-130245.539451
-130245.932161
-130246.18944
-130246.430579
-130246.69159
-130246.955407
-130247.152409
-130247.320459
-130247.523404
-130247.625857
-130247.759093
-130247.865838
-130248.002405
-130248.099485
-130248.196063
-130248.294615
-130248.387782
-130248.438174
-130248.532424
-130248.611011
-130248.667843
-130248.727813
-130248.744072
-130248.858142
-130248.894271
-130248.932862
-130248.964298
-130249.021992
-130249.089286
-130249.108076
-130249.14545
-130249.143231
-130249.199339
-130249.240932
-130249.282016
-130249.287594
-130249.327226
-130249.352015
-130249.364284
-130249.42263
-130249.437064
-130249.466374
-130249.460224
-130249.502135
-130249.524077
-130249.53826
-130249.572733
-130249.569244
-130249.572951


In [ ]:
for i in range(k):
    print("# ", i)
    for t in sorted([(i,rate) for (i,rate) in enumerate(beta.T[i])], key=lambda t: t[1],reverse=True)[:10]:
        print(dictionary[t[0]]," : ",t[1])
    print("\n")

In [ ]:
theta = np.array([np.random.dirichlet(a) for a in alpha])
beta = np.array([np.random.dirichlet(e) for e in eta])

In [ ]:
theta.shape

In [ ]:
beta.shape

In [ ]:
np.inner(theta, beta).shape

In [ ]:
np.log(np.inner(theta, beta)).sum()

In [ ]:
def normliation(q):
    return q/q.sum(axis=0)

In [ ]:
q_theta = normliation(np.array([np.random.dirichlet(g) for g in gamma_]).prod(axis=0))
q_beta = normliation(np.array([np.random.dirichlet(l) for l in lambda_]).prod(axis=1))
q_z = q_[d,w,i].prod()

In [ ]:
q_z

In [ ]:
q_theta.shape

In [ ]:
q_beta.shape

In [ ]:
theta = np.array([np.random.dirichlet(g) for g in gamma_])

In [ ]:
theta

In [ ]:
from scipy import stats


def d_KL(p1,p2):
    return stats.entropy(p1, p2, 2)